In [1]:
import pandas as pd
from script.login import brain_login
from script.alpha import process_alphas_in_parallel
import itertools

In [2]:
# Authenticate with the BRAIN platform
session = brain_login()

Authenticating with WorldQuant BRAIN...
Authentication successful!


In [6]:
# Define your threading and chunking strategy
NUM_THREADS = 1  # Number of parallel workers
CHUNK_SIZE = 2   # Number of individual simulations per API request (max 10)

# --- Define the components for your simulations ---
alpha_expressions = ['close']

# --- NEW: Explicitly build the list of every simulation to run ---
# This gives you full control to create any combination you want.

simulation_list = []
default_settings = {
    "instrumentType": "EQUITY", "delay": 1, 
    "pasteurization": "ON", "unitHandling": "VERIFY", "nanHandling": "OFF",
    "language": "FASTEXPR", "visualization": False, "maxTrade": "OFF",
    "testPeriod": "P1Y"
}

# --- UPDATED: Define parameter lists to create all combinations ---
regions = ["USA"]                                   # e.g., ["USA", "CHN"]
universes = ["TOP1000","TOP3000"]                               # e.g., ["TOP1000", "TOP3000"]
decays = [4]
neutralizations = ["MARKET", "SUBINDUSTRY"]           # e.g., ["SECTOR", "SUBINDUSTRY"]
truncations = [0.08]

# This loop structure will now generate every possible permutation
for expr in alpha_expressions:
    for reg in regions:
        for uni in universes:
            for dec in decays:
                for neut in neutralizations:
                    for trunc in truncations:
                        # Create a fresh copy of settings for this specific simulation
                        settings = default_settings.copy()
                        settings.update({
                            "region": reg,
                            "universe": uni,
                            "decay": dec,
                            "neutralization": neut,
                            "truncation": trunc
                        })
                        
                        # Add the fully-formed simulation object to our master list
                        simulation_list.append({
                            'type': 'REGULAR',
                            'settings': settings,
                            'regular': expr
                        })

print(f"Total simulations to be processed: {len(simulation_list)}")


Total simulations to be processed: 4


In [7]:
if session:
    process_alphas_in_parallel(
        session=session,
        simulation_list=simulation_list,
        num_threads=NUM_THREADS,
        chunk_size=CHUNK_SIZE
    )

Initialized queue with 2 chunks to be processed by 1 workers.
[Worker-1] Processing new chunk of 2 simulation(s)...
[Worker-1] Submission successful. ID: 2s4B1o3YA57dapq70P1ieDq
[Worker-1] Successfully retrieved details for simulation: 2s4B1o3YA57dapq70P1ieDq
[Worker-1] Results successfully saved to: output\TK4Gof.csv
[Worker-1] Processing new chunk of 2 simulation(s)...
[Worker-1] Submission successful. ID: 48BdT68b657fbN3x5ohxc8u
[Worker-1] Successfully retrieved details for simulation: 48BdT68b657fbN3x5ohxc8u
[Worker-1] Results successfully saved to: output\sK7qTN.csv

--- All alpha simulations complete. ---
